In [1]:
import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn
import unicodedata

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder

from scipy.sparse.linalg import svds

import matplotlib.pyplot as plt

In [2]:
from keras.models import Sequential, Model
from keras.layers import Dense, Embedding, Input, Conv2D, Concatenate, MaxPool2D, Reshape, Flatten, Dropout
from keras.layers import ReLU
from keras.layers import BatchNormalization, LSTM, Bidirectional

from keras.callbacks import CSVLogger, EarlyStopping, ReduceLROnPlateau
from keras.layers import Add

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [3]:
from keras.optimizers import Adam, SGD
from keras.callbacks import LearningRateScheduler
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from sklearn.manifold import TSNE
from keras import backend as K

In [4]:
# !python3 -m pip install gensim

In [5]:
from gensim.models import Word2Vec

In [6]:
%matplotlib inline

In [7]:
# !python3 -m pip install -U nltk
# !python3 -m nltk.downloader punkt

In [8]:
import nltk

In [9]:
from nltk.corpus import stopwords

In [11]:
path = 'recipes_with_calories_protein_and_fat.csv'

In [12]:
df = pd.read_csv(path, sep="|", index_col=0)

In [13]:
all_columns = list(df.columns)
len(all_columns)

680

In [14]:
df.dropna(axis=0).shape, df.shape

((15933, 680), (20052, 680))

In [15]:
for column in df.columns:
    if df[column].isnull().any():
        print(column)
#         df[column] = df[column].fillna(df[column].mean())

sodium


In [16]:
df.head()

,title,rating,calories,protein,fat,sodium,#cakeweek,#wasteless,22-minute meals,3-ingredient recipes,...,yellow squash,yogurt,yonkers,yuca,zucchini,cookbooks,leftovers,snack,snack week,turkey
0,"Lentil, Apple, and Turkey Wrap",2.500,426.000000,30.000000,7.000000,559.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,Boudin Blanc Terrine with Red Onion Confit,4.375,403.000000,18.000000,23.000000,1439.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Potato and Fennel Soup Hodge,3.750,165.000000,6.000000,7.000000,165.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Mahi-Mahi in Tomato Olive Sauce,5.000,252.445801,14.630608,23.051666,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Spinach Noodle Casserole,3.125,547.000000,20.000000,32.000000,452.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
def get_small_dict(df, keep_columns, index):
    my_dict = df.iloc[index][keep_columns].to_dict()
    my_dict['tags'] = [
        key for key, value in df.iloc[index].to_dict().items() if value != 0 and key not in keep_columns]
    my_dict['title'] = my_dict['title'].strip()
    my_dict['tags'] = [
        str(unicodedata.normalize('NFKD', t).encode('ascii','ignore'), encoding='ascii') for t in my_dict['tags']
    ]
    return my_dict

In [18]:
keep_columns = ['title', 'rating', 'calories', 'protein', 'fat','sodium']

In [19]:
def remove_outliers_with_tukey(df):
    print('There were {} samples before outlier removal.'.format(len(df)))

    # Remove calories outliers by the Tukey rule
    Q3 = np.percentile(df[regr_column], 75)
    Q1 = np.percentile(df[regr_column], 25)
    IQR = Q3 - Q1

    outliers = df[regr_column].loc[
        (df[regr_column] < (Q1 - 1.5 * IQR)) | 
        (df[regr_column] > (Q3 + 1.5 * IQR))
    ].index
    
    df = df.drop(outliers)
    
    print('There are now {} samples.'.format(len(cut_df)))

    return df

In [20]:
# pick regression column
regr_column = 'sodium'
cut_df = df[df[regr_column] == df[regr_column]]

cut_df = remove_outliers_with_tukey(df=cut_df)

# transform df in list of dictionaries for processing
regr_data = []
for i in range(len(cut_df)):
    regr_data.append(get_small_dict(cut_df, keep_columns, i))

There were 15933 samples before outlier removal.
There are now 15933 samples.


In [21]:
def remove_punctuation_signs(regr_data):
    for d in regr_data:
        if 'tags' not in d:
            raise ValueError('Expected to have <tags> key in each of the regression input data dictionary!')
    corpus = ['|'.join(
        d['tags'] + 
        [w.lower() for w in nltk.word_tokenize(d['title']) if
             w not in '''!()-[]{};:'"\,<>./?@#$%^&*_~''']
    ) for d in regr_data]
    return corpus

In [22]:
# make sure the data doesn't have any punctuation signs
corpus = remove_punctuation_signs(regr_data)

In [23]:
# pick tokenization method
def tokenize(word_list):
    return word_list.split('|')

In [24]:
def get_y_labels(regr_column, regr_data):
    return [d[regr_column] for d in regr_data]

In [25]:
y_label = get_y_labels(regr_column, regr_data)

In [26]:
if len(corpus) != len(y_label):
    print('[ERROR] There is a mismatch between the size of the input data ' +
          ' ({} elements) and the size of labels ({} elements)!'.format(len(corpus), len(y_label)))

## Try a vanilla regression (CountVectorizer + LinearRegression from sklearn)

In [28]:
# fit vectorizer
vectorizer = CountVectorizer(tokenizer=tokenize)
X = vectorizer.fit_transform(corpus)

# train, test data
X_train, X_test, y_train, y_test = train_test_split(X.toarray(), y_label, test_size=0.2, random_state=42)

# sklearn LinearRegression model
regr_sklearn = LinearRegression()
regr_sklearn.fit(X_train, y_train)

# predict data
y_pred = regr_sklearn.predict(X_test)

# how far off is the result? metric = mean squared error
y_test = np.array(y_test)
y_pred = np.array(y_pred)
print("Mean squared error: ", np.sqrt(mean_squared_error(y_true=y_test, y_pred=y_pred)))
print("Std: ", np.std(y_test))
print("Mean: ", np.mean(y_test))

# show first k differences
first_k = 10
for yp, yt in zip(y_pred[:first_k], y_test[:first_k]):
    print(yp, yt)

Mean squared error:  38731625035070.02
Std:  398.30466209532875
Mean:  388.23959390862944
727.6857209718462 807.0
41.72844558122125 13.0
-239326630549.9331 1026.0
327.4044709718467 27.0
194.56658034684625 17.0
168.49480300309625 4.0
286.00310378434625 1277.0
809.6041780030962 802.0
63.05534987809625 47.0
-62620869683.67085 255.0


## Try Word2Vec embeddings + Keras NN

### Word2Vec embedding

In [29]:
vocab = set()
for word_list in corpus:
    vocab |= set(tokenize(word_list))

print("Choose embedding size between {} and {}".format(np.log2(len(vocab)), np.power(len(vocab), 1/4) * 10))

w2v_corpus = []
for word_list in corpus:
    w2v_corpus.append(tokenize(word_list))

Choose embedding size between 12.522091068887065 and 87.57522363959801


In [30]:
EMBEDDING_SIZE = 50

In [31]:
w2v_model = Word2Vec(w2v_corpus, size=EMBEDDING_SIZE, window=5, min_count=1, workers=-1)

print("Embedding size is {}".format(w2v_model.wv.vector_size))

Embedding size is 50


In [32]:
sequence_lengths = [len(w) for w in w2v_corpus]

print("Sequence length should be {} ".format(pd.Series(sequence_lengths).describe()['mean']))

Sequence length should be 17.02348720725599 


In [33]:
FIXED_LENGTH = 17

In [34]:
def generate_embeddings_matrix(sample, model, fixed_length=17):
    embedding_size = model.wv.vector_size
    embeddings = []
    for sen in sample:
        for word in sen:
            if word not in model.wv.vocab:
                continue
            embeddings.append(model.wv[word])
            if len(embeddings) >= fixed_length:
                break

    embeddings = np.array(embeddings)

    while len(embeddings) < fixed_length:
        padding_length = fixed_length - len(embeddings)
        embeddings = np.concatenate((embeddings, embeddings[:padding_length]))
    if len(embeddings) > fixed_length:
        embeddings = embeddings[:fixed_length]

    return embeddings

In [35]:
def generate_input_matrix(sample, fixed_length=17):
    
    inputs = np.array(sample)

    while len(inputs) < fixed_length:
        padding_length = fixed_length - len(inputs)
        inputs = np.concatenate((inputs, inputs[:padding_length]))
    if len(inputs) > fixed_length:
        inputs = inputs[:fixed_length]

    return inputs

In [36]:
# Generate list of input data (embedding matrix) and make sure every input has FIXED_LENGTH elements
X = []
for sample in w2v_corpus:
    embedded_sample = generate_embeddings_matrix(sample, w2v_model, FIXED_LENGTH)
    if embedded_sample.shape != (FIXED_LENGTH, EMBEDDING_SIZE):
        print(embedded_sample.shape)
    X.append(embedded_sample)

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y_label, test_size=0.2, random_state=42)

### Keras NN

In [38]:
FIXED_LENGTH = 17
EMBEDDING_SIZE = 50

BATCH_SIZE = 16
EPOCHS=200

In [39]:
def baseline_model():
    # Create model
    input_layer = Input(shape=(FIXED_LENGTH, EMBEDDING_SIZE), name='input')
    flat = Flatten()(input_layer)
    z = Dense(32, activation='relu', name='fc1', kernel_initializer='normal')(flat)
    output = Dense(1, kernel_initializer='normal')(z)
    model = Model(inputs=[input_layer], outputs=[output])
    # Compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [40]:
model = baseline_model()

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y_label, test_size=0.2, random_state=42)

In [42]:
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [43]:
model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE)

Epoch 1/200
11819/11819 [==============================] - 2s 150us/step - loss: 286549.1342
Epoch 2/200
11819/11819 [==============================] - 1s 115us/step - loss: 219008.3270
Epoch 3/200
11819/11819 [==============================] - 2s 127us/step - loss: 167176.8133
Epoch 4/200
11819/11819 [==============================] - 3s 225us/step - loss: 152465.4664
Epoch 5/200
11819/11819 [==============================] - 2s 169us/step - loss: 150200.9800
Epoch 6/200
11819/11819 [==============================] - 2s 146us/step - loss: 149391.6533
Epoch 7/200
11819/11819 [==============================] - 1s 110us/step - loss: 148751.8980
Epoch 8/200
11819/11819 [==============================] - 1s 107us/step - loss: 148183.8203
Epoch 9/200
11819/11819 [==============================] - 2s 167us/step - loss: 147669.9793
Epoch 10/200
11819/11819 [==============================] - 2s 152us/step - loss: 147217.8869
Epoch 11/200
11819/11819 [==============================] - 2s 145us/

11819/11819 [==============================] - 1s 81us/step - loss: 139868.7170
Epoch 89/200
11819/11819 [==============================] - 1s 78us/step - loss: 139843.4099
Epoch 90/200
11819/11819 [==============================] - 1s 77us/step - loss: 139824.2823
Epoch 91/200
11819/11819 [==============================] - 1s 77us/step - loss: 139797.8746
Epoch 92/200
11819/11819 [==============================] - 1s 82us/step - loss: 139775.9162 0s - loss: 139861.30
Epoch 93/200
11819/11819 [==============================] - 1s 76us/step - loss: 139764.5325
Epoch 94/200
11819/11819 [==============================] - 1s 83us/step - loss: 139725.0942
Epoch 95/200
11819/11819 [==============================] - 1s 76us/step - loss: 139718.0831
Epoch 96/200
11819/11819 [==============================] - 1s 79us/step - loss: 139696.5783
Epoch 97/200
11819/11819 [==============================] - 1s 78us/step - loss: 139670.7842
Epoch 98/200
11819/11819 [==============================] - 1s

11819/11819 [==============================] - 1s 92us/step - loss: 138360.3343
Epoch 175/200
11819/11819 [==============================] - 1s 85us/step - loss: 138341.9087
Epoch 176/200
11819/11819 [==============================] - 1s 81us/step - loss: 138319.2523
Epoch 177/200
11819/11819 [==============================] - 1s 83us/step - loss: 138309.3200
Epoch 178/200
11819/11819 [==============================] - 1s 102us/step - loss: 138265.3092
Epoch 179/200
11819/11819 [==============================] - 1s 83us/step - loss: 138270.9723
Epoch 180/200
11819/11819 [==============================] - 1s 80us/step - loss: 138262.5618
Epoch 181/200
11819/11819 [==============================] - 1s 78us/step - loss: 138236.6922
Epoch 182/200
11819/11819 [==============================] - 1s 81us/step - loss: 138226.5254
Epoch 183/200
11819/11819 [==============================] - 1s 82us/step - loss: 138202.1481
Epoch 184/200
11819/11819 [==============================] - 1s 91us/step

In [44]:
y_pred = model.predict(X_test)

y_pred = y_pred.flatten()

# how far off is the result? metric = mean squared error
y_test = np.array(y_test)
y_pred = np.array(y_pred)
print("Mean squared error: ", np.sqrt(mean_squared_error(y_true=y_test, y_pred=y_pred)))
print("Std: ", np.std(y_test))
print("Mean: ", np.mean(y_test))

# show first k differences
first_k = 10
print("Showing the first {} differences between predicted and real.".format(first_k))
for yp, yt in zip(y_pred[:first_k], y_test[:first_k]):
    print(yp, yt)

Mean squared error:  382.70228388164253
Std:  398.30466209532875
Mean:  388.23959390862944
Showing the first 10 differences between predicted and real.
479.1853 807.0
351.99854 13.0
521.92505 1026.0
493.14148 27.0
303.63733 17.0
416.2398 4.0
417.30725 1277.0
471.766 802.0
518.8888 47.0
476.62866 255.0


## Keras embedding + NN

### Generate input data 

In [45]:
X = []
for sample in w2v_corpus:
    input_sample = generate_input_matrix(sample, FIXED_LENGTH)
    if input_sample.shape != (FIXED_LENGTH,):
        print(input_sample.shape)
    X.append(input_sample)

### Encode the words

In [46]:
encoder = LabelEncoder()
encoder.fit(np.array(X).flatten())

LabelEncoder()

In [47]:
X_encoded = []
for sentence in X:
    encoded = encoder.transform(sentence)
    X_encoded.append(encoded)

### Split X in train and test

In [65]:
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_label, test_size=0.2, random_state=42)

In [66]:
INPUT_SHAPE = (FIXED_LENGTH, )
INPUT_SHAPE

(17,)

### Standardization

In [76]:
y_train = np.array([(y - mean_y)/std_y for y in y_train])

### Define the Keras embedded model 

In [101]:
def keras_embedding_model(encoder):
    # Create model
    input_layer = Input(shape=(FIXED_LENGTH,), name='input')
    embedded_text = Embedding(input_dim=len(encoder.classes_),
                              output_dim=EMBEDDING_SIZE,
                              name='text_embedding')(input_layer)
    embedded_text = Flatten()(embedded_text)
    z = BatchNormalization()(embedded_text)
    z = Dropout(0.3)(z)
    z = Dense(256, activation='relu', name='fc0', kernel_initializer='normal')(z)
    z = BatchNormalization()(z)
    z = Dropout(0.7)(z)
    z = Dense(64, activation='relu', name='fc1', kernel_initializer='normal')(z)
    z = BatchNormalization()(z)
    z = Dropout(0.55)(z)
    z = Dense(16, activation='relu', name='fc2', kernel_initializer='normal')(z)
    z = BatchNormalization()(z)
    z = Dropout(0.4)(z)
    z = ReLU()(z)
    output = Dense(1, kernel_initializer='normal', activation='linear')(z)
    model = Model(inputs=[input_layer], outputs=[output])

    # Compile model
    model.compile(optimizer=Adam(lr=0.01), loss='mean_squared_error')

    return model

In [102]:
keras_embedded_model = keras_embedding_model(encoder=encoder)

In [103]:
keras_embedded_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 17)                0         
_________________________________________________________________
text_embedding (Embedding)   (None, 17, 50)            242400    
_________________________________________________________________
flatten_7 (Flatten)          (None, 850)               0         
_________________________________________________________________
batch_normalization_13 (Batc (None, 850)               3400      
_________________________________________________________________
dropout_11 (Dropout)         (None, 850)               0         
_________________________________________________________________
fc0 (Dense)                  (None, 256)               217856    
_________________________________________________________________
batch_normalization_14 (Batc (None, 256)               1024      
__________

In [95]:
def schedule(epoch, lr):
    if epoch in [500]:
        return lr / 3.0
    return lr
scheduler = LearningRateScheduler(schedule=schedule)

In [96]:
y_series = pd.Series(y_label)
std_y, mean_y = y_series.describe()['std'], y_series.describe()['mean']

def rmse(y_true, y_pred):
    y_true = y_true * std_y + mean_y
    y_pred = y_pred * std_y + mean_y
    return K.sqrt(K.mean(K.square(y_true - y_pred), axis=0))

In [97]:
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [98]:
X_train.shape, y_train.shape

((11819, 17), (11819,))

In [104]:
history = keras_embedded_model.fit(X_train, y_train, epochs=EPOCHS,
                    callbacks=[scheduler, EarlyStopping(patience=10, monitor='rmse'),
                               ModelCheckpoint('models/embedding.h5', monitor='rmse')],
                    batch_size=BATCH_SIZE)

Epoch 1/200
11819/11819 [==============================] - 13s 1ms/step - loss: 0.8176


/home/skadi/.local/lib/python3.5/site-packages/keras/callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `rmse` which is not available. Available metrics are: lr,loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


Epoch 2/200
11819/11819 [==============================] - 29985s 3s/step - loss: 0.6767: 9:26 - loss
Epoch 3/200
11819/11819 [==============================] - 19s 2ms/step - loss: 0.6101
Epoch 4/200
11819/11819 [==============================] - 12s 1ms/step - loss: 0.5804
Epoch 5/200
11819/11819 [==============================] - 10s 847us/step - loss: 0.5356
Epoch 6/200
11819/11819 [==============================] - 11s 889us/step - loss: 0.5218
Epoch 7/200
11819/11819 [==============================] - 11s 889us/step - loss: 0.5057
Epoch 8/200
11819/11819 [==============================] - 11s 967us/step - loss: 0.4783
Epoch 9/200
11819/11819 [==============================] - 11s 918us/step - loss: 0.4633
Epoch 10/200
11819/11819 [==============================] - 11s 920us/step - loss: 0.4613
Epoch 11/200
11819/11819 [==============================] - 11s 914us/step - loss: 0.4434
Epoch 12/200
11819/11819 [==============================] - 11s 933us/step - loss: 0.4356
Epoch 13/

11819/11819 [==============================] - 13s 1ms/step - loss: 0.3397
Epoch 94/200
11819/11819 [==============================] - 10s 834us/step - loss: 0.3316
Epoch 95/200
11819/11819 [==============================] - 10s 863us/step - loss: 0.3344
Epoch 96/200
11819/11819 [==============================] - 10s 815us/step - loss: 0.3354
Epoch 97/200
11819/11819 [==============================] - 13s 1ms/step - loss: 0.3347
Epoch 98/200
11819/11819 [==============================] - 9s 789us/step - loss: 0.3383
Epoch 99/200
11819/11819 [==============================] - 9s 789us/step - loss: 0.3327
Epoch 100/200
11819/11819 [==============================] - 9s 792us/step - loss: 0.3405
Epoch 101/200
11819/11819 [==============================] - 9s 735us/step - loss: 0.3175
Epoch 102/200
11819/11819 [==============================] - 8s 713us/step - loss: 0.3266
Epoch 103/200
11819/11819 [==============================] - 8s 719us/step - loss: 0.3279
Epoch 104/200
11819/11819 [==

Epoch 184/200
11819/11819 [==============================] - 8s 702us/step - loss: 0.3082
Epoch 185/200
11819/11819 [==============================] - 8s 700us/step - loss: 0.3089
Epoch 186/200
11819/11819 [==============================] - 9s 781us/step - loss: 0.3047
Epoch 187/200
11819/11819 [==============================] - 8s 710us/step - loss: 0.3052
Epoch 188/200
11819/11819 [==============================] - 8s 705us/step - loss: 0.2916
Epoch 189/200
11819/11819 [==============================] - 8s 704us/step - loss: 0.3025
Epoch 190/200
11819/11819 [==============================] - 8s 706us/step - loss: 0.3036
Epoch 191/200
11819/11819 [==============================] - 8s 704us/step - loss: 0.3039
Epoch 192/200
11819/11819 [==============================] - 8s 700us/step - loss: 0.3052
Epoch 193/200
11819/11819 [==============================] - 8s 715us/step - loss: 0.3058
Epoch 194/200
11819/11819 [==============================] - 8s 698us/step - loss: 0.3022
Epoch 195/

In [106]:
y_pred = keras_embedded_model.predict(X_test)

y_pred = y_pred.flatten()
y_pred = [y * std_y + mean_y for y in y_pred]

# how far off is the result? metric = mean squared error
y_test = np.array(y_test)
y_pred = np.array(y_pred)
print("Mean squared error: ", np.sqrt(mean_squared_error(y_true=y_test, y_pred=y_pred)))
print("Std: ", np.std(y_test))
print("Mean: ", np.mean(y_test))

# show first k differences
first_k = 100
print("Showing the first {} differences between predicted and real.".format(first_k))
for yp, yt in zip(y_pred[:first_k], y_test[:first_k]):
    print(yp, yt)

Mean squared error:  321.974247937375
Std:  398.30466209532875
Mean:  388.23959390862944
Showing the first 100 differences between predicted and real.
647.8311903670052 807.0
90.61284746943744 13.0
173.18180846909476 1026.0
176.06908392837124 27.0
114.00719159009924 17.0
100.76992958909153 4.0
436.4611931841066 1277.0
659.2225978858446 802.0
126.12242494967501 47.0
365.3802210180413 255.0
507.8161116945199 231.0
142.4604295326618 118.0
262.30197312045857 203.0
187.36148063529518 421.0
79.26294092628945 9.0
850.4921444228644 961.0
780.7301779556489 1109.0
474.57237481470383 725.0
762.0621564896951 35.0
134.4860921684361 26.0
228.84321987337225 21.0
473.5140529877665 561.0
123.81010701222351 2.0
706.2978024785766 848.0
153.72860168816402 2.0
605.4575674627038 848.0
160.80977608572684 49.0
218.20573109732595 377.0
61.96806656214511 1.0
255.76275264074894 233.0
369.26639538572397 340.0
129.5577104646601 122.0
115.79516066160289 12.0
673.0953787872317 514.0
81.42610886399916 13.0
155.979794

### Save model

In [109]:
keras_embedded_model.save('models/predict_sodium')

### Get the dataframe that has NaN values for our regression column (calories)

In [110]:
predict_df = df[df[regr_column] != df[regr_column]]

### Get the input data we need to predict the needed values

In [111]:
predict_data = []
for i in range(len(predict_df)):
    predict_data.append(get_small_dict(predict_df, keep_columns, i))
corpus = ['|'.join(
    d['tags'] + 
    [w.lower() for w in nltk.word_tokenize(d['title']) if w not in '''!()-[]{};:'"\,<>./?@#$%^&*_~''']
) for d in predict_data]

### Split input data using the tokenizer

In [112]:
predict_corpus = []
for word_list in corpus:
    predict_corpus.append(tokenize(word_list))

### Make sure input data has a fixed shape

In [113]:
X = []
for sample in predict_corpus:
    sample = [w for w in sample if w in encoder.classes_]
    input_sample = generate_input_matrix(sample, FIXED_LENGTH)
    if input_sample.shape != (FIXED_LENGTH,):
        print(input_sample.shape)
    X.append(input_sample)

### Transform matrix-like input data using available encoder

In [114]:
X_encoded = []
for sentence in X:
    encoded = encoder.transform(sentence)
    X_encoded.append(encoded)

In [115]:
X_encoded = np.array(X_encoded)
X_encoded.shape

(4119, 17)

### Predict missing data 

In [116]:
y_pred = keras_embedded_model.predict(X_encoded)

y_pred = y_pred.flatten()

### Fill the dataframe with predicted values

In [117]:
predict_df[regr_column] = y_pred

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


### Update initial dataframe with missing values

In [119]:
update_regr_column = [np.nan for _ in range(len(df))]
for idx, value in zip(predict_df.index, y_pred):
    update_regr_column[idx] = value

In [120]:
df.update({regr_column: update_regr_column})

### Save updated dataframe

In [121]:
df.to_csv('recipes_with_calories_protein_fat_and_sodium.csv', sep='|')